In [1]:
from BPNN.fp_calculator import set_sym, db_to_fp
from BPNN.train_agent import Agent
import numpy as np
from ase.db import connect
import torch
import os

### Set fingerprints parameters

In [2]:
# set symm func parameters
elements = ['Pd', 'Au']  # [1, 2 in the param_set]
Gs = [2, 4]
cutoff = 6.0
g2_etas = [0.01, 0.05, 1.0, 4.0]
g2_Rses = [0.0]
g4_etas = [0.03, 0.2]  # if only use g2, set g4 parameters to None
g4_zetas = [1.0, 2.0]
g4_lambdas = [1.0, -1.0]

params_set = set_sym(elements, Gs, cutoff, g2_etas=g2_etas, g2_Rses=g2_Rses, 
                     g4_etas=g4_etas, g4_zetas=g4_zetas, g4_lambdas=g4_lambdas)

# working folder: save the fingerprints, scaling file as well as the model files
Name = 'PdAu-bulk-222'
if not os.path.isdir(f'./{Name}'):
    os.mkdir(f'./{Name}')

### Calculate fingerprints

In [3]:
train_db_name = './db/AuPd-bulk-222-dft-train.db'
valid_db_name = './db/AuPd-bulk-222-dft-valid.db'

# calculate fingerprints for databases, and store them
train_db = connect(train_db_name)
train_data = db_to_fp(train_db, params_set)
torch.save(train_data, f'./{Name}/PdAu_train.sav')

valid_db = connect(valid_db_name)
valid_data = db_to_fp(valid_db, params_set)
torch.save(valid_data, f'./{Name}/PdAu_valid.sav')

### Load fingerprints and start the training process

In [4]:
Name = 'PdAu-bulk-222'

# load data
train_data = torch.load(f'./{Name}/PdAu_train.sav')
valid_data = torch.load(f'./{Name}/PdAu_valid.sav')
test_data = torch.load(f'./{Name}/PdAu_valid.sav')

# specialize log file and model files
element = torch.tensor([46, 79])  # using atomic number to distinguish NNs for different elements
model_paths = [f'./{Name}/model_for_{i}.sav' for i in element.tolist()]
scale_path = f'./{Name}/scale.sav'
log_name = f'./{Name}/log.txt'

# instantiate training agent
device = torch.device('cpu')
agent = Agent(train_data=train_data, valid_data=valid_data, model_paths=model_paths, scale_path=scale_path, 
              test_data=test_data, hidden_1=10, hidden_2=10, lr=1.0, max_iter=10, history_size=100, device=device)

# start training process
agent.train(log_name=log_name, n_epoch=101, interupt=True, val_interval=10, 
            is_force=False, nrg_convg=2.0, force_convg=20, nrg_coef=1000, force_coef=10)

# After training process, we can check the training trajectory in the specified log file 

### Evaluate performance of models

In [6]:
Name = 'PdAu-bulk-222'

# specialize log file and model files
element = torch.tensor([46, 79])  # using atomic number to distinguish NNs for different elements
model_paths = [f'./{Name}/model_for_{i}.sav' for i in element.tolist()]
scale_path = f'./{Name}/scale.sav'
log_name = f'./{Name}/log.txt'

# the evaluation is conducted on test_data of the agent, since we did not specify the test_data above, so we can re-instantiate
# an agent. As an example, we use the same datatest for these three variables since in evaluation step, train_data and valid_data
# don't matter.
train_data = torch.load(f'./{Name}/PdAu_valid.sav')
valid_data = torch.load(f'./{Name}/PdAu_valid.sav')
test_data = torch.load(f'./{Name}/PdAu_valid.sav')

# instantiate training agent
device = torch.device('cpu')
agent = Agent(train_data=train_data, valid_data=valid_data, model_paths=model_paths, scale_path=scale_path, 
              test_data=test_data, hidden_1=10, hidden_2=10, lr=1.0, max_iter=10, history_size=100, device=device)

# test function returns energy_predictions, force_predictions, energy_mae, force_mae in order.
# if is_force = False, return zeros for force_predctions and force mae
nrg_pred, forces_pred, nrg_mae, force_mae = agent.test(is_force=False)
print(f'{nrg_mae*1000} meV/atom')

1.5829404592514038 meV/atom
